In [1]:
import sys
import os.path as op
sys.path.append('..')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import geopandas as gpd

import parkingadvisor as pa

In [3]:
DATA = op.join(pa.__path__[0], 'data')
RAW_DATA = op.join(pa.__path__[0], 'data', 'raw_data')

# 1. Clean Annual_Parking_Data

Read a .csv file as a DataFrame with specific columns and drop the columns containing missing data.


In [4]:
# import "Annual_Parking_Study_Data" to creat 'pkdt' dataframe
# read data
col_names = ['Date Time', 'Unitdesc', 'Side', 'Parking_Spaces', 'Total_Vehicle_Count', 'Construction', 'Event Closure']
pkdt = pa.read_data(RAW_DATA + '\Annual_Parking_Study_Data.csv', col_names)
pkdt = pkdt.dropna()

pkdt.head(10)

C:\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


,Date Time,Unitdesc,Side,Parking_Spaces,Total_Vehicle_Count,Construction,Event Closure
0,4-08-14 14:00:00,12TH AVE BETWEEN E COLUMBIA ST AND E MARION ST,W,8.0,9.0,No,No
1,4-08-14 15:00:00,12TH AVE BETWEEN E COLUMBIA ST AND E MARION ST,W,8.0,8.0,No,No
2,4-08-14 13:00:00,12TH AVE BETWEEN E COLUMBIA ST AND E MARION ST,W,8.0,6.0,No,No
3,4-08-14 09:00:00,12TH AVE BETWEEN E COLUMBIA ST AND E MARION ST,W,8.0,6.0,No,No
4,4-08-14 12:00:00,12TH AVE BETWEEN E COLUMBIA ST AND E MARION ST,W,8.0,6.0,No,No
5,4-08-14 16:00:00,12TH AVE BETWEEN E COLUMBIA ST AND E MARION ST,W,8.0,7.0,No,No
6,4-08-14 13:00:00,12TH AVE BETWEEN E CHERRY ST AND E COLUMBIA ST,W,7.0,7.0,No,No
7,4-08-14 15:00:00,12TH AVE BETWEEN E CHERRY ST AND E COLUMBIA ST,E,6.0,5.0,No,No
8,4-08-14 17:00:00,12TH AVE BETWEEN E CHERRY ST AND E COLUMBIA ST,E,6.0,6.0,No,No
9,4-08-14 16:00:00,12TH AVE BETWEEN E MARION ST AND E SPRING ST,W,11.0,11.0,No,No


Drop the exceptions under construction and event closure.

In [5]:
#filter the data with 'Construction' and 'Event' column are "No"
values = ['Yes', 'yes', 'ERROR: #N/A']
pkdt = pa.data_filter(pkdt, 'Construction', values)
pkdt = pa.data_filter(pkdt, 'Event Closure', ['Yes'])
pkdt = pa.data_filter(pkdt, 'Parking_Spaces',[0])

pkdt = pkdt.drop('Construction', axis = 1)
pkdt = pkdt.drop('Event Closure', axis = 1)

Calculate the average occupancy of each street per hour of the day

In [6]:
#group rows with same 'Time' and 'Unitdesc', for other columns, find the sum
pkdt = pkdt.groupby(['Date Time', 'Unitdesc'], as_index = False)['Parking_Spaces', 'Total_Vehicle_Count'].sum()

pkdt.head(10)

,Date Time,Unitdesc,Parking_Spaces,Total_Vehicle_Count
0,3-03-15 08:00:00,11TH AVE NE BETWEEN NE 43RD ST AND NE 45TH ST,14.0,16.0
1,3-03-15 08:00:00,11TH AVE NE BETWEEN NE 45TH ST AND NE 47TH ST,28.0,18.0
2,3-03-15 08:00:00,11TH AVE NE BETWEEN NE 47TH ST AND NE 50TH ST,52.0,32.0
3,3-03-15 08:00:00,12TH AVE NE BETWEEN NE 43RD ST AND NE 45TH ST,60.0,48.0
4,3-03-15 08:00:00,12TH AVE NE BETWEEN NE 45TH ST AND NE 47TH ST,38.0,4.0
5,3-03-15 08:00:00,12TH AVE NE BETWEEN NE 47TH ST AND NE 50TH ST,34.0,4.0
6,3-03-15 08:00:00,15TH AVE NE BETWEEN NE 42ND ST AND NE 43RD ST,10.0,0.0
7,3-03-15 08:00:00,15TH AVE NE BETWEEN NE 45TH ST AND NE 47TH ST,32.0,0.0
8,3-03-15 08:00:00,15TH AVE NE BETWEEN NE 47TH ST AND NE 50TH ST,48.0,14.0
9,3-03-15 08:00:00,BROOKLYN AVE NE BETWEEN NE 41ST ST AND NE 42ND ST,68.0,8.0


In [7]:
#find the occupancy and append it to the pkdt
Occupancy = np.minimum(pkdt['Total_Vehicle_Count'] / pkdt['Parking_Spaces'], 1)
pkdt['Occupancy'] = Occupancy
pkdt = pkdt.dropna()
pkdt = pkdt.drop('Total_Vehicle_Count', axis = 1)

Convert time `String` into `Datetime` 

In [8]:
#retrivev 'hour' column from general datetime
pkdt['Date Time'] = pd.to_datetime(pkdt['Date Time'], format = '%m-%d-%y %H:%M:%S')
times = pd.DatetimeIndex(pkdt['Date Time'])
pkdt['Hour'] = times.hour
#for 'Hour' column, replace 0 with 24
pkdt = pkdt.replace({'Hour': 0}, 24)

pkdt.head(10)

,Date Time,Unitdesc,Parking_Spaces,Occupancy,Hour
0,2015-03-03 08:00:00,11TH AVE NE BETWEEN NE 43RD ST AND NE 45TH ST,14.0,1.000000,8
1,2015-03-03 08:00:00,11TH AVE NE BETWEEN NE 45TH ST AND NE 47TH ST,28.0,0.642857,8
2,2015-03-03 08:00:00,11TH AVE NE BETWEEN NE 47TH ST AND NE 50TH ST,52.0,0.615385,8
3,2015-03-03 08:00:00,12TH AVE NE BETWEEN NE 43RD ST AND NE 45TH ST,60.0,0.800000,8
4,2015-03-03 08:00:00,12TH AVE NE BETWEEN NE 45TH ST AND NE 47TH ST,38.0,0.105263,8
5,2015-03-03 08:00:00,12TH AVE NE BETWEEN NE 47TH ST AND NE 50TH ST,34.0,0.117647,8
6,2015-03-03 08:00:00,15TH AVE NE BETWEEN NE 42ND ST AND NE 43RD ST,10.0,0.000000,8
7,2015-03-03 08:00:00,15TH AVE NE BETWEEN NE 45TH ST AND NE 47TH ST,32.0,0.000000,8
8,2015-03-03 08:00:00,15TH AVE NE BETWEEN NE 47TH ST AND NE 50TH ST,48.0,0.291667,8
9,2015-03-03 08:00:00,BROOKLYN AVE NE BETWEEN NE 41ST ST AND NE 42ND ST,68.0,0.117647,8


In [9]:
#group data by 'hour' and 'Unidesc' and average other columns
pkdt = pkdt.groupby(['Hour', 'Unitdesc'], as_index = False)['Parking_Spaces', 'Occupancy'].mean()

pkdt.head(10)

,Hour,Unitdesc,Parking_Spaces,Occupancy
0,1,10TH AVE BETWEEN E PIKE ST AND E PINE ST,40.0,0.350000
1,1,10TH AVE BETWEEN E UNION ST AND E PIKE ST,20.0,0.550000
2,1,11TH AVE BETWEEN E PIKE ST AND E PINE ST,29.0,0.586207
3,1,12TH AVE BETWEEN E MADISON ST AND E PIKE ST,18.0,0.388889
4,1,12TH AVE BETWEEN E PIKE ST AND E PINE ST,3.0,1.000000
5,1,13TH AVE BETWEEN E MADISON ST AND E PIKE ST,5.0,1.000000
6,1,13TH AVE BETWEEN E PIKE ST AND E PINE ST,12.0,0.500000
7,1,14TH AVE BETWEEN E PIKE ST AND E PINE ST,9.0,0.444444
8,1,15TH AVE BETWEEN E MADISON ST AND E PINE ST,4.0,1.000000
9,1,BELMONT AVE BETWEEN E PIKE ST AND E PINE ST,8.0,1.000000


Get the prelim streets flow datafile.

In [10]:
# Rename the column names to all_capital
aps = pkdt.rename(index=str, columns={"Hour": "HOUR", "Unitdesc": "UNITDESC",
                                'Parking_Spaces':"PARKING_SPACE", 
                                'Occupancy': "OCCUPANCY"})

# 2. Clean blockface

Make a subset of blockface.csv containing only the streets in Annual_Parking_Study.csv

In [11]:
#import "Blockface" to creat 'bfdt' dataframe
#read data
col_names = ['UNITDESC','WKD_RATE1','WKD_START1', 'WKD_END1', 'WKD_RATE2', 'WKD_START2', 'WKD_END2',
                      'WKD_RATE3', 'WKD_START3', 'WKD_END3', 'SAT_RATE1', 'SAT_START1', 'SAT_END1',
                     'SAT_RATE2', 'SAT_START2', 'SAT_END2', 'SAT_RATE3', 'SAT_START3', 'SAT_END3',
                     'PARKING_TIME_LIMIT']
bfdt = pa.read_data(RAW_DATA + '\Blockface.csv', col_names)

bfdt.head(10)

C:\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (34,36,37,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


,UNITDESC,WKD_RATE1,WKD_START1,WKD_END1,WKD_RATE2,WKD_START2,WKD_END2,WKD_RATE3,WKD_START3,WKD_END3,SAT_RATE1,SAT_START1,SAT_END1,SAT_RATE2,SAT_START2,SAT_END2,SAT_RATE3,SAT_START3,SAT_END3,PARKING_TIME_LIMIT
0,S DEARBORN ST BETWEEN 7TH AVE S AND 8TH AVE S,0.5,480,659,2.0,660,1019,1.5,1020.0,1199.0,0.5,480.0,659.0,2.0,660.0,1019.0,1.5,1020.0,1199.0,NaN
1,2ND AVE N BETWEEN JOHN ST AND THOMAS ST,1.0,480,659,1.5,660,1019,1.5,1020.0,1199.0,1.0,480.0,659.0,1.5,660.0,1019.0,1.5,1020.0,1199.0,NaN
2,NE CAMPUS EB PY BETWEEN 12TH AVE NE AND BROOKL...,2.0,480,659,2.5,660,1019,2.5,1020.0,1199.0,2.0,480.0,659.0,2.5,660.0,1019.0,2.5,1020.0,1199.0,NaN
3,5TH AVE BETWEEN PIKE ST AND PINE ST,3.5,480,659,4.0,660,1019,3.5,1020.0,1199.0,3.5,480.0,659.0,4.0,660.0,1019.0,3.5,1020.0,1199.0,NaN
4,OCCIDENTAL AVE S BETWEEN RAILROAD WAY S AND S ...,2.5,480,659,5.0,660,1079,NaN,NaN,NaN,2.5,480.0,659.0,5.0,660.0,1079.0,NaN,NaN,NaN,30.0
5,STEWART ST BETWEEN 8TH AVE AND 9TH AVE,1.5,480,659,2.5,660,1079,NaN,NaN,NaN,1.5,480.0,659.0,2.5,660.0,1079.0,NaN,NaN,NaN,NaN
6,ALASKAN WAY BETWEEN WALL ST AND VINE ST,1.0,480,659,2.0,660,1019,1.5,1020.0,1199.0,1.0,480.0,659.0,2.0,660.0,1019.0,1.5,1020.0,1199.0,NaN
7,S MAIN ST BETWEEN 4TH AVE S AND 5TH AVE S,0.5,480,659,2.0,660,1019,1.5,1020.0,1199.0,0.5,480.0,659.0,2.0,660.0,1019.0,1.5,1020.0,1199.0,30.0
8,7TH AVE BETWEEN OLIVE WAY AND STEWART ST,3.5,480,659,4.5,660,1019,3.0,1020.0,1199.0,3.5,480.0,659.0,4.5,660.0,1019.0,3.0,1020.0,1199.0,120.0
9,SENECA ST BETWEEN 1ST AVE AND 2ND AVE,3.5,480,659,4.0,660,1019,3.5,1020.0,1199.0,3.5,480.0,659.0,4.0,660.0,1019.0,3.5,1020.0,1199.0,120.0


Change all end time to correct format, i.e. 1199 --> 1200

In [12]:
end_col = ['WKD_END1', 'WKD_END2', 'WKD_END3', 'SAT_END1', 'SAT_END2', 'SAT_END3']
# modify the end time of all rates        
pa.modify_end_time(bfdt, end_col)

bfdt.head(10)

,UNITDESC,WKD_RATE1,WKD_START1,WKD_END1,WKD_RATE2,WKD_START2,WKD_END2,WKD_RATE3,WKD_START3,WKD_END3,SAT_RATE1,SAT_START1,SAT_END1,SAT_RATE2,SAT_START2,SAT_END2,SAT_RATE3,SAT_START3,SAT_END3,PARKING_TIME_LIMIT
0,S DEARBORN ST BETWEEN 7TH AVE S AND 8TH AVE S,0.5,480,660,2.0,660,1020,1.5,1020.0,1200.0,0.5,480.0,660.0,2.0,660.0,1020.0,1.5,1020.0,1200.0,NaN
1,2ND AVE N BETWEEN JOHN ST AND THOMAS ST,1.0,480,660,1.5,660,1020,1.5,1020.0,1200.0,1.0,480.0,660.0,1.5,660.0,1020.0,1.5,1020.0,1200.0,NaN
2,NE CAMPUS EB PY BETWEEN 12TH AVE NE AND BROOKL...,2.0,480,660,2.5,660,1020,2.5,1020.0,1200.0,2.0,480.0,660.0,2.5,660.0,1020.0,2.5,1020.0,1200.0,NaN
3,5TH AVE BETWEEN PIKE ST AND PINE ST,3.5,480,660,4.0,660,1020,3.5,1020.0,1200.0,3.5,480.0,660.0,4.0,660.0,1020.0,3.5,1020.0,1200.0,NaN
4,OCCIDENTAL AVE S BETWEEN RAILROAD WAY S AND S ...,2.5,480,660,5.0,660,1080,NaN,NaN,NaN,2.5,480.0,660.0,5.0,660.0,1080.0,NaN,NaN,NaN,30.0
5,STEWART ST BETWEEN 8TH AVE AND 9TH AVE,1.5,480,660,2.5,660,1080,NaN,NaN,NaN,1.5,480.0,660.0,2.5,660.0,1080.0,NaN,NaN,NaN,NaN
6,ALASKAN WAY BETWEEN WALL ST AND VINE ST,1.0,480,660,2.0,660,1020,1.5,1020.0,1200.0,1.0,480.0,660.0,2.0,660.0,1020.0,1.5,1020.0,1200.0,NaN
7,S MAIN ST BETWEEN 4TH AVE S AND 5TH AVE S,0.5,480,660,2.0,660,1020,1.5,1020.0,1200.0,0.5,480.0,660.0,2.0,660.0,1020.0,1.5,1020.0,1200.0,30.0
8,7TH AVE BETWEEN OLIVE WAY AND STEWART ST,3.5,480,660,4.5,660,1020,3.0,1020.0,1200.0,3.5,480.0,660.0,4.5,660.0,1020.0,3.0,1020.0,1200.0,120.0
9,SENECA ST BETWEEN 1ST AVE AND 2ND AVE,3.5,480,660,4.0,660,1020,3.5,1020.0,1200.0,3.5,480.0,660.0,4.0,660.0,1020.0,3.5,1020.0,1200.0,120.0


In [13]:
# group rows by same 'UNITDESC', average other columns
bfdt = bfdt.groupby(['UNITDESC'], as_index = False).mean()


In [14]:
# convert minutes to datetime
col_names = ['WKD_START1', 'WKD_END1', 'WKD_START2', 'WKD_END2',
             'WKD_START3', 'WKD_END3', 'SAT_START1', 'SAT_END1',
             'SAT_START2', 'SAT_END2', 'SAT_START3', 'SAT_END3']
pa.convert_datetime_to_h(bfdt, col_names)

bfdt.head()

,UNITDESC,WKD_RATE1,WKD_START1,WKD_END1,WKD_RATE2,WKD_START2,WKD_END2,WKD_RATE3,WKD_START3,WKD_END3,SAT_RATE1,SAT_START1,SAT_END1,SAT_RATE2,SAT_START2,SAT_END2,SAT_RATE3,SAT_START3,SAT_END3,PARKING_TIME_LIMIT
0,10TH AVE BETWEEN E ALDER ST AND E TERRACE ST,0.0,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,10TH AVE BETWEEN E FIR ST AND DEAD END 1,0.0,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,10TH AVE BETWEEN E MADISON ST AND E SENECA ST,2.0,8,11,3.0,11,17,3.0,17.0,22.0,2.0,8.0,11.0,3.0,11.0,17.0,3.0,17.0,22.0,120.0
3,10TH AVE BETWEEN E PIKE ST AND E PINE ST,2.0,8,11,3.0,11,17,3.0,17.0,22.0,2.0,8.0,11.0,3.0,11.0,17.0,3.0,17.0,22.0,120.0
4,10TH AVE BETWEEN E SENECA ST AND E UNION ST,2.0,8,11,3.0,11,17,3.0,17.0,22.0,2.0,8.0,11.0,3.0,11.0,17.0,3.0,17.0,22.0,180.0


In [15]:
# select the blockface only including in APS dataset
bfdt_only_APS = pa.subset([bfdt, 'UNITDESC'], [aps, 'UNITDESC'])

Read the dataset we got in step 1. Compare the number of streets.

In [16]:
# The number of streets in APS
print('============BEDORE===========')
print('Streets in APS: {}\nStreets in Blockface: {}'.format(len(aps.UNITDESC.unique()), len(bfdt_only_APS.UNITDESC.unique())))

============BEDORE===========
Streets in APS: 1245
Streets in Blockface: 1234


Some streets are not included in Blockface.csv
Drop these from APS file.

In [17]:
# Save the subset as .csv file
# bfdt_only_APS.to_csv(DATA + '\Rate_limit.csv')

In [18]:
aps = pa.subset([aps, 'UNITDESC'], [bfdt_only_APS, 'UNITDESC'])

In [19]:
# Check the number of streets
print('============AFTER===========')
print('Streets in APS: {}\nStreets in Blockface: {}'.format(len(aps.UNITDESC.unique()),len(bfdt_only_APS.UNITDESC.unique())))

============AFTER===========
Streets in APS: 1234
Streets in Blockface: 1234


In [20]:
# Save the final occupancy file.
# aps.to_csv(DATA + '\occupancy_per_hour.csv')

# 3. Clean Streets GIS file

Make a subset of the streets only containing in above files.

In [21]:
# Read the gis data of Seattle streets
data = gpd.read_file(RAW_DATA + '\Seattle_Streets\Seattle_Streets.shp')

In [22]:
# Filter the gis data by parking study
street_geo = pa.subset([data,'UNITDESC'], [aps, 'UNITDESC'])

In [23]:
# Save the dataframe as a GeoJSON file
# pa.convert_to_geojson(street_geo, DATA + '\Streets_gis.json')